In [ ]:
%%writefile matrixadd.cu
#include <stdio.h>

#define N 5
#define M 6

__global__ void init(int* matrix1,int* matrix2){
unsigned id=threadIdx.y * blockDim.x + threadIdx.x;
matrix1[id]=id;
matrix2[id]=2*id;
}

__global__ void addition(int* matrix1,int* matrix2,int* result){
unsigned id=threadIdx.y * blockDim.x + threadIdx.x;
result[id]=matrix1[id]+matrix2[id];
}
int main(){
  int* gpumatrix1,*gpumatrix2,*resultmatrix;
  int* cpumatrix;

  cudaMalloc((void**)&gpumatrix1,N*M*sizeof(int));
  cudaMalloc((void**)&gpumatrix2,N*M*sizeof(int));
  cudaMalloc((void**)&resultmatrix,N*M*sizeof(int));

  cpumatrix=(int*)malloc(N*M*sizeof(int));

  dim3 block(N,M,1);

  init<<<1,block>>> (gpumatrix1,gpumatrix2);
  addition<<<1,block>>> (gpumatrix1,gpumatrix2,resultmatrix);

  cudaMemcpy(cpumatrix,resultmatrix,N*M*sizeof(int),cudaMemcpyDeviceToHost);

  for(int i=0;i<N;++i){
    for(int j=0;j<M;++j){
      printf("%d ",cpumatrix[i*M+j]);
    }
    printf("\n");
  }

  cudaFree(gpumatrix1);
  cudaFree(gpumatrix2);
  cudaFree(resultmatrix);
  free(cpumatrix);

}

Overwriting matrixadd.cu


In [ ]:
!nvcc matrixadd.cu -o m -arch=sm_75 -Xcompiler -Wall


In [ ]:
!./m

0 3 6 9 12 15 
18 21 24 27 30 33 
36 39 42 45 48 51 
54 57 60 63 66 69 
72 75 78 81 84 87 


In [ ]:
%%writefile matrixmul.cu
#include <stdio.h>
#include <stdlib.h>

#define N 180

__global__ void init(int* matrix1, int* matrix2) {
   /* unsigned id = threadIdx.y * blockDim.x + threadIdx.x;
    if (id < N * N) {
        matrix1[id] = id;
        matrix2[id] = 2 * id;
    }
    */
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < N && col < N) {
        matrix1[row * N + col] = row * N+ col;
        matrix2[row * N + col] = 2 *(row * N + col);
    }
}

__global__ void kernel(int* matrix1, int* matrix2, int* result) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        int sum = 0;
        for (int i = 0; i < N; ++i) {
            sum += matrix1[row * N + i] * matrix2[i * N + col];
        }
        result[row * N + col] = sum;
    }
}

int main() {
    int* gpumatrix1, *gpumatrix2, *resultmatrix, *cpumatrix,*cpumatrix1,*cpumatrix2;
    cpumatrix = (int*)malloc(N * N * sizeof(int));
    cpumatrix1 = (int*)malloc(N * N * sizeof(int));
    cpumatrix2 = (int*)malloc(N * N * sizeof(int));

    cudaMalloc((void**)&gpumatrix1, N * N * sizeof(int));
    cudaMalloc((void**)&gpumatrix2, N * N * sizeof(int));
    cudaMalloc((void**)&resultmatrix, N * N * sizeof(int));

    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid((N + 15) / 16, (N + 15) / 16);

    init<<<blocksPerGrid, threadsPerBlock>>>(gpumatrix1, gpumatrix2);
    cudaDeviceSynchronize();
    cudaMemcpy(cpumatrix1, gpumatrix1, N * N * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(cpumatrix2, gpumatrix2, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    printf("Printing gpu matrix 1\n");
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%4d ", cpumatrix1[i * N + j]);
        }
        printf("\n");
    }

    printf("Printing gpu matrix 2\n");
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%4d ", cpumatrix2[i * N + j]);
        }
        printf("\n");
    }

    kernel<<<blocksPerGrid, threadsPerBlock>>>(gpumatrix1, gpumatrix2, resultmatrix);
    cudaDeviceSynchronize();

    cudaMemcpy(cpumatrix, resultmatrix, N * N * sizeof(int), cudaMemcpyDeviceToHost);
    printf("Printing result matrix \n");

    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%4d ", cpumatrix[i * N + j]);
        }
        printf("\n");
    }

    cudaFree(gpumatrix1);
    cudaFree(gpumatrix2);
    free(cpumatrix1);
    free(cpumatrix2);

    cudaFree(resultmatrix);
    free(cpumatrix);

    return 0;
}


Writing matrixmul.cu


In [ ]:
!nvcc matrixmul.cu -o m -arch=sm_75 -Xcompiler -Wall


In [ ]:
!./m

Printing gpu matrix 1
   0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49   50   51   52   53   54   55   56   57   58   59   60   61   62   63   64   65   66   67   68   69   70   71   72   73   74   75   76   77   78   79   80   81   82   83   84   85   86   87   88   89   90   91   92   93   94   95   96   97   98   99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179 
 180  181  182  183  184  185  186  187  188  189  190  191  192  193  194  1

In [ ]:
%%writefile matix_transpose.cu

#include <stdio.h>
#include <stdlib.h>

#define N 18 // Rows
#define M 19// Columns

__global__ void init(int* matrix) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < N && col < M) {
        matrix[row * M + col] = row * M + col;
    }
}

__global__ void transpose(int* input, int* output) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < N && col < M) {
        output[col * N + row] = input[row * M + col];
    }
}

int main() {
    int* cpumatrix = (int*)malloc(N * M * sizeof(int));
    int* cputranspose = (int*)malloc(N * M * sizeof(int));
    int* gpumatrix;
    int* gputranspose;

    cudaMalloc((void**)&gpumatrix, N * M * sizeof(int));
    cudaMalloc((void**)&gputranspose, N * M * sizeof(int));

    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid((M + 15) / 16, (N + 15) / 16);

    init<<<blocksPerGrid, threadsPerBlock>>>(gpumatrix);
    cudaDeviceSynchronize();

    cudaMemcpy(cpumatrix, gpumatrix, N * M * sizeof(int), cudaMemcpyDeviceToHost);
    printf("Original Matrix (%dx%d):\n", N, M);
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < M; ++j) {
            printf("%4d ", cpumatrix[i * M + j]);
        }
        printf("\n");
    }

    transpose<<<blocksPerGrid, threadsPerBlock>>>(gpumatrix, gputranspose);
    cudaDeviceSynchronize();

    cudaMemcpy(cputranspose, gputranspose, N * M * sizeof(int), cudaMemcpyDeviceToHost);
    printf("\nTransposed Matrix (%dx%d):\n", M, N);
    for (int i = 0; i < M; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%4d ", cputranspose[i * N + j]);
        }
        printf("\n");
    }

    cudaFree(gpumatrix);
    cudaFree(gputranspose);
    free(cpumatrix);
    free(cputranspose);

    return 0;
}


Writing matix_transpose.cu


In [ ]:
!nvcc matix_transpose.cu -o m -arch=sm_75 -Xcompiler -Wall

In [ ]:
!./m

Original Matrix (18x19):
   0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   18 
  19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37 
  38   39   40   41   42   43   44   45   46   47   48   49   50   51   52   53   54   55   56 
  57   58   59   60   61   62   63   64   65   66   67   68   69   70   71   72   73   74   75 
  76   77   78   79   80   81   82   83   84   85   86   87   88   89   90   91   92   93   94 
  95   96   97   98   99  100  101  102  103  104  105  106  107  108  109  110  111  112  113 
 114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132 
 133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151 
 152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170 
 171  172  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189 
 190  191  192 

In [ ]:
%%writefile matix_transpose.cu

#include <stdio.h>
#include <stdlib.h>

#define N 25

__global__ void init(int* matrix) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < N && col < N) {
        matrix[row * N + col] = row * N + col;

    }
}

__global__ void transpose(int* input) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < N && col < N) {
        if(col>row){
          int temp=input[col*N+row];
          input[col * N + row] = input[row * N + col];
          input[row * N + col] = temp;
        }
    }
}

int main() {
    int* cpumatrix = (int*)malloc(N * N * sizeof(int));
    int* gpumatrix;

    cudaMalloc((void**)&gpumatrix, N * N * sizeof(int));

    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid((N + 15) / 16, (N + 15) / 16);

    init<<<blocksPerGrid, threadsPerBlock>>>(gpumatrix);
    cudaDeviceSynchronize();

    cudaMemcpy(cpumatrix, gpumatrix, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    printf("Original Matrix (%dx%d):\n", N, N);
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%4d ", cpumatrix[i * N + j]);
        }
        printf("\n");
    }

    transpose<<<blocksPerGrid, threadsPerBlock>>>(gpumatrix);

    cudaDeviceSynchronize();
    cudaMemcpy(cpumatrix, gpumatrix, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    printf("\nTransposed Matrix (%dx%d):\n", N, N);
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%4d ", cpumatrix[i * N + j]);
        }
        printf("\n");
    }

    cudaFree(gpumatrix);
    free(cpumatrix);

    return 0;
}


Overwriting matix_transpose.cu


In [ ]:
!nvcc matix_transpose.cu -o m -arch=sm_75 -Xcompiler -Wall

In [ ]:
!./m

Original Matrix (25x25):
   0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24 
  25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49 
  50   51   52   53   54   55   56   57   58   59   60   61   62   63   64   65   66   67   68   69   70   71   72   73   74 
  75   76   77   78   79   80   81   82   83   84   85   86   87   88   89   90   91   92   93   94   95   96   97   98   99 
 100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124 
 125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149 
 150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174 
 175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  192  19

In [ ]:
%%writefile isol.cu

#include <stdio.h>
#define N 4

__device__ int no_of_isolated_nodes=0;
__device__ int isolated_nodes[N];

__global__ void kernel(int* adjmatrix){
int row=blockIdx.x*blockDim.x + threadIdx.x;
if(row<N){
int count=0;

for(int col=0;col<N;++col){
if(adjmatrix[row*N+ col]==0) count++;
}
if(count==N){
int index = atomicAdd(&no_of_isolated_nodes, 1);
isolated_nodes[index] = row;
}
}
}

int main(){
  int cpumatrix[4][4]={{0,1,1,0},{0,0,0,0},{1,1,1,0},{0,0,0,0}};
  int* gpumatrix;
  cudaMalloc(&gpumatrix,N*N*sizeof(int));
  cudaMemcpy(gpumatrix,cpumatrix,N*N*sizeof(int),cudaMemcpyHostToDevice);
  kernel<<<1,N>>>(gpumatrix);

  int no_of_isolated_nodes_cpu;
  int isolated_nodes_cpu[N];
  cudaMemcpyFromSymbol(&no_of_isolated_nodes_cpu,no_of_isolated_nodes,sizeof(int));
  cudaMemcpyFromSymbol(&isolated_nodes_cpu,isolated_nodes,N*sizeof(int));

  printf(" No of isolated nodes are: %d \n",no_of_isolated_nodes_cpu);
  printf("The isolated nodes are: \n");
  for(int i=0;i<no_of_isolated_nodes_cpu;++i){
    printf("%d ",isolated_nodes_cpu[i]);
  }
}

Overwriting isol.cu


In [ ]:
!nvcc isol.cu -o m -arch=sm_75 -Xcompiler -Wall

In [ ]:
!./m

 No of isolated nodes are: 2 
The isolated nodes are: 
1 3 

In [ ]:
%%writefile symmetric.cu

#include <stdio.h>
#define N 18

__device__ int symmetric=0;

__global__ void kernel(int* adjmatrix){

int id=blockIdx.x*blockDim.x+ threadIdx.x;

int row=id/N;
int col=id%N;

if(col>row && adjmatrix[row*N+col]==adjmatrix[col*N+row]){
atomicAdd(&symmetric,1);
}
}

int main(){
  int cpumatrix[N][N]={
    {0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0},
    {1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0},
    {0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0},
    {0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0},
    {0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0},
    {0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0},
    {0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0},
    {0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0},
    {1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0},
    {0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0},
    {0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0},
    {0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0},
    {0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0},
    {0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0},
    {0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0},
    {0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0},
    {0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1},
    {0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0}};

    int* gpumatrix;
    cudaMalloc(&gpumatrix,N*N*sizeof(int));
    cudaMemcpy(gpumatrix,cpumatrix,N*N*sizeof(int),cudaMemcpyHostToDevice);
    int blocksize=16;
    int gridsize=(N*N+blocksize-1)/blocksize;
    kernel<<<gridsize,blocksize>>>(gpumatrix);
    int ans;
    cudaMemcpyFromSymbol(&ans,symmetric,sizeof(int));
    if(ans==(N*(N-1)/2)) printf("This is a symmetric graph\n");

    printf("%d",ans);

}

Writing symmetric.cu


In [ ]:
!nvcc symmetric.cu -o m -arch=sm_75 -Xcompiler -Wall

In [ ]:
!./m

152

In [ ]:
%%writefile self_loops.cu
#include <stdio.h>
#define N 4

__device__ int no_of_self_loops=0;

__global__ void kernel(int* adjmatrix){
  int i=blockIdx.x*blockDim.x+threadIdx.x;
  if(id<N){  if(adjmatrix[i*N+i]==1) atomicAdd(&no_of_self_loops,1);}
}


int main(){
  int cpumatrix[N][N]={{1,0,0,0},{1,1,0,0},{1,0,0,0},{1,0,0,0}};
  int* gpumatrix;
  cudaMalloc(&gpumatrix,N*N*sizeof(int));
  cudaMemcpy(gpumatrix,cpumatrix,N*N*sizeof(int),cudaMemcpyHostToDevice);
  kernel<<<(N+2)/3,3>>>(gpumatrix);
  int ans;
cudaMemcpyFromSymbol(&ans,no_of_self_loops,sizeof(int));
printf("%d",ans);
cudaFree(gpumatrix);
}


Overwriting self_loops.cu


In [ ]:
!nvcc self_loops.cu -o m -arch=sm_75 -Xcompiler -Wall

In [ ]:
!./m

2

In [ ]:
%%writefile edge_to_adj_matrix.cu
#define E 6
#define N 4

__global__ void kernel(int* matrix,int* edges){
int id=blockIdx.x*blockDim.x+threadIdx.x;
if(id<E){
  int u=edges[id*2+0];
  int v=edges[id*2+1];
  matrix[u*N+v]=1;
  matrix[v*N+u]=1;
}

}

#include <stdio.h>
int main(){
      int edges[E][2] = {{0,1}, {0,2}, {1,2}, {2,0}, {2,3}, {3,3}};
      int adjmatrix[N][N]={0};

      int* gpumatrix;
      int* gpuedges;

      cudaMalloc(&gpumatrix,N*N*sizeof(int));
      cudaMalloc(&gpuedges,E*2*sizeof(int));

      cudaMemcpy(gpumatrix,adjmatrix,N*N*sizeof(int),cudaMemcpyHostToDevice);
      cudaMemcpy(gpuedges,edges,E*2*sizeof(int),cudaMemcpyHostToDevice);

      kernel<<<1,E>>> (gpumatrix,gpuedges);

      cudaMemcpy(adjmatrix,gpumatrix,N*N*sizeof(int),cudaMemcpyDeviceToHost);

      for(int i=0;i<N;++i){
        for(int j=0;j<N;++j){
          printf("%d ",adjmatrix[i][j]);
        }
        printf("\n");
      }

      cudaFree(gpumatrix);
      cudaFree(gpuedges);

}

Writing edge_to_adj_matrix.cu


In [ ]:
!nvcc edge_to_adj_matrix.cu -o m -arch=sm_75 -Xcompiler -Wall

In [ ]:
!./m

0 1 1 0 
1 0 1 0 
1 1 0 1 
0 0 1 1 


In [ ]:
#define BLOCKSIZE 1024
__global__ void dkernel(unsigned * vector,unsigned vectorsize){
unsigned id=blockIdx.x*blockDim.x+threadIdx.x;
if(id<vectorsize)
vector[id]=id;
}

int main(int nn,char* str[]){
unsigned N=atoi(str[1]);
// no of data elements is N
unsigned *vector,*hvector;
cudaMalloc(&vector,N*sizeof(unsigned));
hvector=(unsigned*) malloc(N*sizeof(unsigned));
unsigned nblocks=ceil(float)N(/BLOCKSIZE);
//here you are applying ceil to the integer division that is truncated
away
printf("nblocks is %d\n",nblocks);
dkernel<<<nblocks,BLOCKSIZE>>> (vector);
//technically we could also have N blocks of 1 thread each but it leads
to poor performance. we want our bus(analogy) to be as full as possible.
cudaMemcpy(hvector,vector,N*sizeof(unsigned),cudaMemcpyDeviceToHost);
for(unsigned ii=0;ii<N;++ii){
printf("%d",hvector[ii]);
}

In [1]:
%%writefile bfs.cu

#include <stdio.h>
#define N 7

__global__ void kernel(int* visited,int* frontier,int frontier_size,int* g_next_size,int* g_next_frontier,int* level,int* curr_level,int *matrix){
  int id=blockIdx.x*blockDim.x+threadIdx.x;
  if(id<frontier_size){
    int u=frontier[id];
    for(int i=0;i<N;++i){
       if (matrix[u * N + i] == 1 && visited[i] == 0) {
        if (atomicExch(&visited[i], 1) == 0) {
        int pos=atomicAdd(g_next_size,1);
        g_next_frontier[pos]=i;
        level[i]=*curr_level+1;
      }
    }
  }
}
}

int main() {
    // Adjacency matrix of the graph
    int adj_matrix[N][N] = {
        {0, 1, 0, 1, 0, 0, 0},
        {1, 0, 1, 0, 1, 0, 0},
        {0, 1, 0, 0, 0, 0, 0},
        {1, 0, 0, 0, 1, 1, 0},
        {0, 1, 0, 1, 0, 0, 0},
        {0, 0, 0, 1, 0, 0, 1},
        {0, 0, 0, 0, 0, 1, 0}
    };
int* gpumatrix;
cudaMalloc(&gpumatrix,N*N*sizeof(int));
cudaMemcpy(gpumatrix,adj_matrix,N*N*sizeof(int),cudaMemcpyHostToDevice);

int* g_visited;
int visited[N]={0};
visited[0]=1;
cudaMalloc(&g_visited,N*sizeof(int));
cudaMemcpy(g_visited,visited,N*sizeof(int),cudaMemcpyHostToDevice);


int frontier[N];
frontier[0]=0;
visited[0]=1;
int frontier_size=1;

int* g_frontier;
cudaMalloc(&g_frontier,N*sizeof(int));
cudaMemcpy(g_frontier,frontier,N*sizeof(int),cudaMemcpyHostToDevice);

/*int next_frontier[N];*/
int* g_next_frontier;
cudaMalloc(&g_next_frontier,N*sizeof(int));

//int next_frontier_size;
int* g_next_frontier_size;
cudaMalloc(&g_next_frontier_size,sizeof(int));

int level[N]={0};
int* g_level;
cudaMalloc(&g_level,N*sizeof(int));
cudaMemcpy(g_level,level,N*sizeof(int),cudaMemcpyHostToDevice);

int curr_level=0;
int* g_curr_level;
cudaMalloc(&g_curr_level,sizeof(int));
cudaMemcpy(g_curr_level,&curr_level,sizeof(int),cudaMemcpyHostToDevice);

while(frontier_size>0){

kernel<<<1, N>>>(g_visited, g_frontier, frontier_size, g_next_frontier_size, g_next_frontier, g_level, g_curr_level, gpumatrix);
cudaDeviceSynchronize();
cudaMemcpy(&curr_level,g_curr_level,sizeof(int),cudaMemcpyDeviceToHost);
curr_level++;
cudaMemcpy(g_curr_level,&curr_level,sizeof(int),cudaMemcpyHostToDevice);

//now next_frontier becomes frontier

cudaMemcpy(frontier, g_next_frontier, N * sizeof(int), cudaMemcpyDeviceToHost);
cudaMemcpy(g_frontier, g_next_frontier, N * sizeof(int), cudaMemcpyDeviceToDevice);
cudaMemcpy(&frontier_size, g_next_frontier_size, sizeof(int), cudaMemcpyDeviceToHost);
int zero = 0;
//next_frontier_size = 0;
cudaMemcpy(g_next_frontier_size, &zero, sizeof(int), cudaMemcpyHostToDevice);

}

cudaMemcpy(level,g_level,N*sizeof(int),cudaMemcpyDeviceToHost);

for(int i=0;i<N;++i){
  printf("Node %d: Level :%d\n",i,level[i]);
}

//cleanup

cudaFree(gpumatrix);
cudaFree(g_visited);
cudaFree(g_frontier);
cudaFree(g_next_frontier);
cudaFree(g_next_frontier_size);
cudaFree(g_level);
cudaFree(g_curr_level);

}


Writing bfs.cu


In [2]:
!nvcc bfs.cu -o m -arch=sm_75 -Xcompiler -Wall

In [3]:
!./m

Node 0: Level :0
Node 1: Level :1
Node 2: Level :2
Node 3: Level :1
Node 4: Level :2
Node 5: Level :2
Node 6: Level :3


In [17]:
%%writefile bfs.cu
#include <stdio.h>
#include <cuda.h>

#define N 7  // Number of nodes

__global__ void kernel(int *adj_matrix, int *frontier, int frontier_size,
                       int *visited, int *next_frontier, int *next_size, int level, int *levels) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < frontier_size) {
        int u = frontier[idx];
        for (int v = 0; v < N; v++) {
            if (adj_matrix[u * N + v] == 1 && !visited[v]) {
                if (atomicExch(&visited[v], 1) == 0) {
                    int pos = atomicAdd(next_size, 1);
                    next_frontier[pos] = v;
                    levels[v] = level + 1;
                }
            }
        }
    }
}

int main() {
    int h_adj_matrix[N][N] = {
        {0, 1, 0, 1, 0, 0, 0},
        {1, 0, 1, 0, 1, 0, 0},
        {0, 1, 0, 0, 0, 0, 0},
        {1, 0, 0, 0, 1, 1, 0},
        {0, 1, 0, 1, 0, 0, 0},
        {0, 0, 0, 1, 0, 0, 1},
        {0, 0, 0, 0, 0, 1, 0}
    };

    int *d_adj_matrix, *d_frontier, *d_next_frontier, *d_next_size, *d_levels;
    int *d_visited;

    // Allocate device memory
    cudaMalloc(&d_adj_matrix, N * N * sizeof(int));
    cudaMalloc(&d_frontier, N * sizeof(int));
    cudaMalloc(&d_next_frontier, N * sizeof(int));
    cudaMalloc(&d_next_size, sizeof(int));
    cudaMalloc(&d_visited, N * sizeof(int));
    cudaMalloc(&d_levels, N * sizeof(int));

    // Host-side tracking arrays
    int frontier[N] = {0}; // Start from node 0
    int visited[N] = {0};
    int levels[N] = {-1};

    frontier[0] = 0; // Starting from node 0
    visited[0] = 1;
    levels[0] = 0;
    int frontier_size = 1;
    int level = 0;

    // Copy initial data to device
    cudaMemcpy(d_adj_matrix, h_adj_matrix, N * N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_visited, visited, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_frontier, frontier, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_levels, levels, N * sizeof(int), cudaMemcpyHostToDevice);

    while (frontier_size > 0) {
        int zero = 0;
        cudaMemcpy(d_next_size, &zero, sizeof(int), cudaMemcpyHostToDevice);

        int blockSize = 128;
        int numBlocks = (frontier_size + blockSize - 1) / blockSize;

        kernel<<<numBlocks, blockSize>>>(d_adj_matrix, d_frontier, frontier_size,
                                         d_visited, d_next_frontier, d_next_size, level, d_levels);
        cudaDeviceSynchronize();

        cudaMemcpy(&frontier_size, d_next_size, sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(frontier, d_next_frontier, frontier_size * sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(d_frontier, frontier, frontier_size * sizeof(int), cudaMemcpyHostToDevice);

        level++;
    }

    // Copy final levels
    cudaMemcpy(levels, d_levels, N * sizeof(int), cudaMemcpyDeviceToHost);

    printf("BFS levels:\n");
    for (int i = 0; i < N; i++) {
        printf("Node %d -> Level %d\n", i, levels[i]);
    }

    // Free device memory
    cudaFree(d_adj_matrix);
    cudaFree(d_frontier);
    cudaFree(d_next_frontier);
    cudaFree(d_next_size);
    cudaFree(d_visited);
    cudaFree(d_levels);

    return 0;
}


Overwriting bfs.cu


In [18]:
!nvcc bfs.cu -o m -arch=sm_75 -Xcompiler -Wall

In [19]:
!./m

BFS levels:
Node 0 -> Level 0
Node 1 -> Level 1
Node 2 -> Level 2
Node 3 -> Level 1
Node 4 -> Level 2
Node 5 -> Level 2
Node 6 -> Level 3
